# Simple Implementation of IDA*

## Dependencies

In [2]:
import numpy as np
import json
from datetime import datetime

## Read Data

In [3]:
def string2duration(string):
    ''' "01:50:19.3177493" to duration in seconds'''
    date =  datetime.strptime(string.split('.')[0], "%H:%M:%S")
    return date.second + 60*date.minute + 3600*date.hour

In [4]:
def read_data(path):
    global task_count
    global data
    global tasks
    file = open(path)
    data = json.load(file)
    tasks = data['nodes']
    for task in tasks:
        tasks[task]['Data'] = string2duration(tasks[task]['Data'])
    task_count = len(tasks)

## Node Class

In [5]:
class Node():
    def __init__(self, id = '', cost = 0):
        self.children = []
        self.parents = []
        self.id = id
        self.cost = cost
    
    def successors(self):
        return sorted(self.children, key=lambda n: h(n))
    
    def is_goal(self):
        return self.id == 'GOAL'

## Graph Class

In [6]:
class Graph():
    def __init__(self, graph):
        self.nodes = {}
        self.roots = []
        self.load(graph)
        print(list(self.nodes)[-1])
    
    def load(self, graph):
        for id in graph.keys():
            try:
                node = self.nodes[id]
            except:
                node = Node(id, graph[id]['Data'])
            
            if len(graph[id]['Dependencies']) == 0:
                node.isTerminal = False
                self.roots.append(node)
            else:
                for parent in graph[id]['Dependencies']:
                    try:
                        parent_node = self.nodes[str(parent)]
                    except:
                        self.nodes[str(parent)] = Node(str(parent), graph[str(parent)]['Data'])
                        parent_node = self.nodes[str(parent)]
                        
                    parent_node.children.append(node)
                    node.parents.append(parent_node)

            self.nodes[id] = node

## Heuristic

In [7]:
def h(n):
    return 1

## Search

In [8]:
def search(path, g, bound, goal):
    node = path[-1]
    f = g + h(node)
    if f > bound:
        return f
    if node.id == goal:
        return [node.id]
    minimum = np.inf
    for succ in node.successors():
        if succ not in path:
            path.append(succ)
            t = search(path, g + succ.cost, bound, goal)
            if isinstance(t, list):
                return [node.id] + t
            elif t < minimum:
                minimum = t
            path.pop()
    return minimum

## IDA*

In [9]:
def ida_star(root, goal):
    bound = np.inf
    path = [root]
    t = search(path, root.cost, bound, goal)
    if isinstance(t, list):
        return t
    return 'Not found'

## Load Data

In [10]:
read_data("Graphs/smallComplex.json")

## Load Graph

In [11]:
graph = Graph(tasks)

3538


## Launch IDA*

In [12]:
path = ida_star(graph.roots[0], '3538')
print(path)

['2365', '3532', '3538']
